# Generate Fv Sequences using Annotated Inputs for Antigens

In [1]:
from peft import PeftModel, PeftConfig
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

/home/azureuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## Load model
model_name = "../models/peleke-phi-4-h100-20250810"

config = PeftConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, torch_dtype=torch.bfloat16, trust_remote_code=True).cuda()
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, model_name).cuda()

model.eval()

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 211.36it/s]


In [ ]:
## Format Prompt with Tags
def format_prompt(antigen_sequence):
    epitope_seq = re.sub(r'\[([A-Z])\]', r'<epi>\1</epi>', antigen_sequence)
    formatted_str = f"Antigen: {epitope_seq}<|im_end|>\nAntibody:"
    return formatted_str


In [10]:
## List of Antigens to Test
test_antigens = [
    ## PD-1
    "NPPTFSPALLVVTEGDNATFTCSFS[S][F][V]L[N]WYRMQ[T][D][K]LAAF[P]E[D][R][S][Q][P][G]QDSRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGA[I]S[L]AQIKESLRAELRV",
]

In [16]:
## Test Tokenization
# prompt = format_prompt(test_antigens[0])
# tokenizer.tokenize(prompt)

In [19]:
## Generate complete antibody sequences for the example antigens
for antigen in test_antigens:
    prompt = format_prompt(antigen)
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=False,
        )
    
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    antibody_sequence = full_text.split('<|im_end|>')[1].replace('Antibody: ', '')
    print(f"Antigen: {antigen}\nAntibody: {antibody_sequence}\n")
    # print(full_text)

Antigen: NPPTFSPALLVVTEGDNATFTCSFS[S][F][V]L[N]WYRMQ[T][D][K]LAAF[P]E[D][R][S][Q][P][G]QDSRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGA[I]S[L]AQIKESLRAELRV
Antibody: QVQLVQSGGGVVQPGRSLRLSCAASGFTFSSYAMHWVRQAPGKGLEWVAVISYDGSNKYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKSGRSGALDYWGQGTLVTVS|DIQMTQSPSSLSASVGDRVTITCRASQSVSSAVAWYQQKPGKAPKLLIYSASSLYSGVPSRFSGSRSGTDFTLTISSLQPEDFATYYCQQSYSTPRTFGQGTKVEIK

